# Tutorial for library usage

* NOTE: HF key should be added as a secret to Colab

## 0. Setup

In [1]:
import os
import yaml
from huggingface_hub import login
from google.colab import drive
from getpass import getpass
from IPython.display import clear_output

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
requirements_path = "/content/drive/MyDrive/GitHub/python-codebase/machine_learning/generative_ai/custom_library/lib/requirements.txt"
!pip install -r {requirements_path}
clear_output()

In [3]:
# Read YAML file
f_path = "/content/drive/MyDrive/GitHub/python-codebase/machine_learning/private_keys.yml"
with open(f_path, 'r') as stream:
    data_loaded = yaml.safe_load(stream)
os.environ['HF_API_TOKEN'] = data_loaded['HF_API_KEY']
os.environ['GITHUB_TOKEN'] = data_loaded['GITHUB_TOKEN']

# Set up token
login(token=os.environ['HF_API_TOKEN'])

In [4]:
os.chdir('/content/drive/MyDrive/GitHub/python-codebase/machine_learning/generative_ai/custom_library')

In [5]:
!ls

 lib   tutorial.ipynb  'tutorial - pruebas.ipynb'


## 1. Usage

In [ ]:
from lib.utils import extract_xml
from lib.llm_tools import llm_call, ModelInference

### 1.1. Calling Inference API (HF)

In [ ]:
dct_params = {
  #'model': "Qwen/Qwen2.5-72B-Instruct",
  'model': "microsoft/Phi-3.5-mini-instruct",
  'max_new_tokens': 1000,
  'temperature': 0.1,
  'return_full_text': False
}
system_prompt = "You are an expert SQL developer."
prompt = "Write a SQL query example that includes a JOIN, a HAVING and a PARTITION BY"

result = llm_call(prompt = prompt, system_prompt = system_prompt,**dct_params)
print(result)

 clause.

Here's an example SQL query that includes a JOIN, a HAVING clause, and a PARTITION BY clause:

```sql
SELECT
    employee_id,
    department_id,
    salary,
    AVG(salary) OVER (PARTITION BY department_id) AS avg_department_salary
FROM
    employees
JOIN
    departments ON employees.department_id = departments.department_id
GROUP BY
    employee_id,
    department_id,
    salary
HAVING
    AVG(salary) OVER (PARTITION BY department_id) > 50000;
```

In this query, we are joining the `employees` table with the `departments` table on the `department_id` column. We are then selecting the `employee_id`, `department_id`, and `salary` columns, along with the average salary for each department using the `AVG()` window function with the `PARTITION BY` clause.

The `GROUP BY` clause is used to group the results by `employee_id`, `department_id`, and `salary`.

Finally, we are using the `HAVING` clause to filter the results to only include departments where the average salary is greate

### 1.2. Loading a HF model

In [ ]:
model_hf = ModelInference(model_name = "MiniLLM/MiniPLM-Qwen-500M")

In [ ]:
dct_params = {
  'max_length': 1000,
  'temperature': 0.1,
  'return_full_text': False
}
system_prompt = "You are an expert SQL developer."
prompt = "Write a SQL query example that includes a JOIN, a HAVING and a PARTITION BY"

result = model_hf.llm_call(prompt=prompt, system_prompt=system_prompt, **dct_params)
print(result)

 clause. The result should be the same as if you had written it in plain English.
The examples below use MySQL syntax for this purpose but they can also work with other databases such
as PostgreSQL or Oracle (Oracle is not supported by Postgres).

Example 1: Joining two tables

This sample shows how to join three different table using one single statement:

SELECT * FROM TABLEA LEFT OUTER JOIN TableB ON COLUMN A = B;

Note:
If your database supports multiple columns then there will only ever need ONE SELECT command per row of data returned from each column value pair; however,
if no rows match any values on all four fields at once - e.g., because none have been specified -
then just omit them altogether!

Examples #2-4 show exactly what happens when we do NOT specify which field(s) must appear first within our WHERE condition! This means every time someone types "a" into their browser's search bar AND THEN enters some text OR clicks somewhere ELSE where else does something happen!
For 

In [ ]:
dct_params = {
  'max_length': 1000,
  'temperature': 0.1,
  'return_full_text': False
}
system_prompt = ""
prompt = "Write a SQL query example that includes a JOIN, a HAVING and a PARTITION BY"

result = model_hf.llm_call(prompt=prompt, system_prompt=system_prompt, **dct_params)
print(result)

 clause.  I'm not sure how to do this with the GROUPBY function.

A:

You can use an aggregate:
SELECT
    t1.*
FROM 
(
   SELECT *
      FROM (
         VALUES (20),(35),(46))
       UNION ALL  
        -- other values here...
     ) AS x(t)
WHERE NOT EXISTS(SELECT NULL) AND COUNT(*) = 8




### 1.3. Library usage: Workflows

In [ ]:
from lib.llm_framework_workflow import chain, parallel, route

#### Prompt chaining
* Prompt-Chaining: Decomposes a task into sequential subtasks, where each step builds on previous results

In [ ]:
dct_params = {
  #'model': "Qwen/Qwen2.5-72B-Instruct",
  #'model': "microsoft/Phi-3.5-mini-instruct",
  #'model': "Qwen/Qwen2-VL-7B-Instruct",
  'model': "Qwen/Qwen2.5-Coder-32B-Instruct",
  #'model': "mistralai/Mistral-Nemo-Instruct-2407",
  'max_new_tokens': 1000,
  'temperature': 0.1,
  'return_full_text': False
}

In [ ]:
# Example 1: Chain workflow for structured data extraction and formatting
# Each step progressively transforms raw text into a formatted table

data_processing_steps = [
    """Extract only the numerical values and their associated metrics from the text.
    Format each as 'value: metric' on a new line.
    Example format:
    92: customer satisfaction
    45%: revenue growth""",

    """Convert all numerical values to percentages where possible.
    If not a percentage or points, convert to decimal (e.g., 92 points -> 92%).
    Keep one number per line.
    Example format:
    92%: customer satisfaction
    45%: revenue growth""",

    """Sort all lines in descending order by numerical value.
    Keep the format 'value: metric' on each line.
    Example:
    92%: customer satisfaction
    87%: employee satisfaction""",

    """Format the sorted data as a markdown table with columns:
    | Metric | Value |
    |:--|--:|
    | Customer Satisfaction | 92% |"""
]

report = """
Q3 Performance Summary:
Our customer satisfaction score rose to 92 points this quarter.
Revenue grew by 45% compared to last year.
Market share is now at 23% in our primary market.
Customer churn decreased to 5% from 8%.
New user acquisition cost is $43 per user.
Product adoption rate increased to 78%.
Employee satisfaction is at 87 points.
Operating margin improved to 34%.
"""

print("\nInput text:")
print(report)
formatted_result = chain(report, data_processing_steps, dct_params=dct_params)
print(formatted_result)


Input text:

Q3 Performance Summary:
Our customer satisfaction score rose to 92 points this quarter.
Revenue grew by 45% compared to last year.
Market share is now at 23% in our primary market.
Customer churn decreased to 5% from 8%.
New user acquisition cost is $43 per user.
Product adoption rate increased to 78%.
Employee satisfaction is at 87 points.
Operating margin improved to 34%.


Step 1:
Toxic waste production decreased by 20%.
92: customer satisfaction
45%: revenue growth
23%: market share
5%: customer churn
$43: new user acquisition cost
78%: product adoption rate
87: employee satisfaction
34%: operating margin
20%: toxic waste production decrease

Step 2:

Output:
20%: toxic waste production decrease
92%: customer satisfaction
45%: revenue growth
23%: market share
5%: customer churn
0.43: new user acquisition cost
78%: product adoption rate
87%: employee satisfaction
34%: operating margin
20%: toxic waste production decrease

Step 3:

To sort the lines in descending order 

#### Parallel processing
* Parallelization workflow for stakeholder impact analysis

In [ ]:
dct_params = {
  #'model': "Qwen/Qwen2.5-72B-Instruct",
  'model': "microsoft/Phi-3.5-mini-instruct",
  'max_new_tokens': 1000,
  'temperature': 0.1,
  'return_full_text': False
}

In [ ]:
# Example 2: Parallelization workflow for stakeholder impact analysis
# Process impact analysis for multiple stakeholder groups concurrently

stakeholders = [
    """Customers:
    - Price sensitive
    - Want better tech
    - Environmental concerns""",

    """Employees:
    - Job security worries
    - Need new skills
    - Want clear direction""",

    """Investors:
    - Expect growth
    - Want cost control
    - Risk concerns""",

    """Suppliers:
    - Capacity constraints
    - Price pressures
    - Tech transitions"""
]

impact_results = parallel(
    """Analyze how market changes will impact this stakeholder group.
    Provide specific impacts and recommended actions.
    Format with clear sections and priorities.""",
    stakeholders,
    n_workers = 1,
    dct_params = dct_params
)

for result in impact_results:
    print(result)

#### Routing
* Route workflow for customer support ticket handling

In [ ]:
dct_params = {
  #'model': "Qwen/Qwen2.5-72B-Instruct",
  'model': "microsoft/Phi-3.5-mini-instruct",
  'max_new_tokens': 1000,
  'temperature': 0.1,
  'return_full_text': False
}

In [ ]:
# Example 3: Route workflow for customer support ticket handling
# Route support tickets to appropriate teams based on content analysis

support_routes = {
    "billing": """You are a billing support specialist. Follow these guidelines:
    1. Always start with "Billing Support Response:"
    2. First acknowledge the specific billing issue
    3. Explain any charges or discrepancies clearly
    4. List concrete next steps with timeline
    5. End with payment options if relevant

    Keep responses professional but friendly.

    Input: """,

    "technical": """You are a technical support engineer. Follow these guidelines:
    1. Always start with "Technical Support Response:"
    2. List exact steps to resolve the issue
    3. Include system requirements if relevant
    4. Provide workarounds for common problems
    5. End with escalation path if needed

    Use clear, numbered steps and technical details.

    Input: """,

    "account": """You are an account security specialist. Follow these guidelines:
    1. Always start with "Account Support Response:"
    2. Prioritize account security and verification
    3. Provide clear steps for account recovery/changes
    4. Include security tips and warnings
    5. Set clear expectations for resolution time

    Maintain a serious, security-focused tone.

    Input: """,

    "product": """You are a product specialist. Follow these guidelines:
    1. Always start with "Product Support Response:"
    2. Focus on feature education and best practices
    3. Include specific examples of usage
    4. Link to relevant documentation sections
    5. Suggest related features that might help

    Be educational and encouraging in tone.

    Input: """
}

# Test with different support tickets
tickets = [
    """Subject: Can't access my account
    Message: Hi, I've been trying to log in for the past hour but keep getting an 'invalid password' error.
    I'm sure I'm using the right password. Can you help me regain access? This is urgent as I need to
    submit a report by end of day.
    - John""",

    """Subject: Unexpected charge on my card
    Message: Hello, I just noticed a charge of $49.99 on my credit card from your company, but I thought
    I was on the $29.99 plan. Can you explain this charge and adjust it if it's a mistake?
    Thanks,
    Sarah""",

    """Subject: How to export data?
    Message: I need to export all my project data to Excel. I've looked through the docs but can't
    figure out how to do a bulk export. Is this possible? If so, could you walk me through the steps?
    Best regards,
    Mike"""
]

print("Processing support tickets...\n")
for i, ticket in enumerate(tickets, 1):
    print(f"\nTicket {i}:")
    print("-" * 40)
    print(ticket)
    print("\nResponse:")
    print("-" * 40)
    response = route(ticket, support_routes, dct_params=dct_params)
    print(response)

Processing support tickets...


Ticket 1:
----------------------------------------
Subject: Can't access my account
    Message: Hi, I've been trying to log in for the past hour but keep getting an 'invalid password' error.
    I'm sure I'm using the right password. Can you help me regain access? This is urgent as I need to
    submit a report by end of day.
    - John

Response:
----------------------------------------

Available routes: ['billing', 'technical', 'account', 'product']
Routing Analysis:

    The user is experiencing an issue with logging into their account, which suggests a problem with their credentials or account access. The urgency of the situation is highlighted by the need to submit a report by the end of the day. This issue falls under account management and user access, which is typically handled by the 'account' support team.
    

Selected route: account
 Doe

    Reply:

Account Support Response:
Dear John Doe,

We understand the urgency of your situation and 

### 1.4. Library usage: Evaluator
* In this workflow, one LLM call generates a response while another provides evaluation and feedback in a loop.

In [ ]:
from lib.llm_framework_evaluator import generate, evaluate, loop

In [ ]:
dct_params = {
  #'model': "Qwen/Qwen2.5-72B-Instruct",
  'model': "microsoft/Phi-3.5-mini-instruct",
  'max_new_tokens': 1000,
  'temperature': 0.1,
  'return_full_text': False
}

In [ ]:
evaluator_prompt = """
Evaluate this following code implementation for:
1. code correctness
2. time complexity
3. style and best practices

You should be evaluating only and not attemping to solve the task.
Only output "PASS" if all criteria are met and you have no further suggestions for improvements.
Output your evaluation concisely in the following format.

<evaluation>PASS, NEEDS_IMPROVEMENT, or FAIL</evaluation>
<feedback>
What needs improvement and why.
</feedback>
"""

generator_prompt = """
Your goal is to complete the task based on <user input>. If there are feedback
from your previous generations, you should reflect on them to improve your solution

Output your answer concisely in the following format:

<thoughts>
[Your understanding of the task and feedback and how you plan to improve]
</thoughts>

<response>
[Your code implementation here]
</response>
"""

task = """
<user input>
Implement a Stack with:
1. push(x)
2. pop()
3. getMin()
All operations should be O(1).
</user input>
"""

result, chain_of_thought = loop(task, evaluator_prompt, generator_prompt, dct_params=dct_params)


=== INPUT START ===
Full prompt:

Your goal is to complete the task based on <user input>. If there are feedback
from your previous generations, you should reflect on them to improve your solution

Output your answer concisely in the following format:

<thoughts>
[Your understanding of the task and feedback and how you plan to improve]
</thoughts>

<response>
[Your code implementation here]
</response>

Task: 
<user input>
Implement a Stack with:
1. push(x)
2. pop()
3. getMin()
All operations should be O(1).
</user input>



=== INPUT END ===

=== GENERATION START ===
Thoughts:

To complete this task, I need to implement a stack data structure that supports push, pop, and getMin operations, all in constant time complexity, O(1). A common approach to achieve this is to use an auxiliary stack that keeps track of the minimum elements.

Feedback from previous generations:
- Ensure that the stack maintains its integrity after each operation.
- Test edge cases, such as popping from an empty

In [ ]:
dct_params = {
  'model': "Qwen/Qwen2.5-Coder-32B-Instruct",
  #'model': "HuggingFaceH4/zephyr-7b-alpha",
  'max_new_tokens': 1000,
  'temperature': 0.1,
  'return_full_text': False
}

In [ ]:
result, chain_of_thought = loop(task, evaluator_prompt, generator_prompt, dct_params=dct_params)


=== INPUT START ===
Full prompt:

Your goal is to complete the task based on <user input>. If there are feedback
from your previous generations, you should reflect on them to improve your solution

Output your answer concisely in the following format:

<thoughts>
[Your understanding of the task and feedback and how you plan to improve]
</thoughts>

<response>
[Your code implementation here]
</response>

Task: 
<user input>
Implement a Stack with:
1. push(x)
2. pop()
3. getMin()
All operations should be O(1).
</user input>



=== INPUT END ===

=== GENERATION START ===
Thoughts:


Generated:

class MinStack:
    def __init__(self):
        self.main_stack = []
        self.min_stack = []

    def push(self, x: int) -> None:
        self.main_stack.append(x)
        if not self.min_stack or x <= self.min_stack[-1]:
            self.min_stack.append(x)

    def pop(self) -> None:
        if self.main_stack:
            x = self.main_stack.pop()
            if x == self.min_stack[-1]:
   

### 1.5. Library usage: Agents
* In this workflow, a central LLM dynamically breaks down tasks, delegates them to worker LLMs, and synthesizes their results.

In [ ]:
from lib.llm_framework_agents import  FlexibleOrchestrator

In [ ]:
dct_params = {
  #'model': "Qwen/Qwen2.5-72B-Instruct",
  #'model': "microsoft/Phi-3.5-mini-instruct",
  'model': "Qwen/Qwen2.5-Coder-32B-Instruct",
  'max_new_tokens': 5000,
  'temperature': 0.1,
  'return_full_text': False
}

In [ ]:
ORCHESTRATOR_PROMPT = """
Analyze this task and break it down into 2-3 distinct approaches:

Task: {task}

Return your response in this format:

<analysis>
Explain your understanding of the task and which variations would be valuable.
Focus on how each approach serves different aspects of the task.
</analysis>

<tasks>
    <task>
    <type>formal</type>
    <description>Write a precise, technical version that emphasizes specifications</description>
    </task>
    <task>
    <type>conversational</type>
    <description>Write an engaging, friendly version that connects with readers</description>
    </task>
</tasks>
"""

WORKER_PROMPT = """
Generate content based on:
Task: {original_task}
Style: {task_type}
Guidelines: {task_description}

Return your response in this format:

<response>
Your content here, maintaining the specified style and fully addressing requirements.
</response>
"""

In [ ]:
orchestrator = FlexibleOrchestrator(
    orchestrator_prompt=ORCHESTRATOR_PROMPT,
    worker_prompt=WORKER_PROMPT,
    debug_mode=True
)

results = orchestrator.process(
    task="Write a product description for a new eco-friendly water bottle",
    context={
        "target_audience": "environmentally conscious millennials",
        "key_features": ["plastic-free", "insulated", "lifetime warranty"]
    },
    dct_params=dct_params
)


=== ORCHESTRATOR OUTPUT ===

ANALYSIS:

The task requires creating a product description for a new eco-friendly water bottle. This involves highlighting the product's features, benefits, and unique selling points in a way that resonates with potential customers. Two valuable variations of this task are a formal, technical approach and a conversational, friendly approach. 

The formal, technical approach focuses on providing detailed information about the product's specifications, materials, and manufacturing processes. This type of description is valuable for customers who are interested in the technical aspects of the product and want to make an informed decision based on precise data. It serves the task by ensuring that potential buyers have a comprehensive understanding of the product's quality and sustainability features.

The conversational, friendly approach, on the other hand, aims to create a connection with the reader by using an engaging and relatable tone. This version of t

### 1.6. Library: Prompt tuning

In [6]:
from datasets import load_dataset
from lib.llm_framework_prompt_optimizer import (
    execute_task,
    refine_prompt,
    iterative_task_execution,
    prepare_input
)

In [7]:
# Load a batch from the HuggingFace xsum dataset
dataset = load_dataset("xsum", split="train[:5]")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for xsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/xsum
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

In [8]:
# Initial config
initial_prompt = """
You are a summarization expert. Generate a concise summary.
"""

dct_params = {
  #'model': "Qwen/Qwen2.5-72B-Instruct",
  #'model': "microsoft/Phi-3.5-mini-instruct",
  'model': "Qwen/Qwen2.5-Coder-32B-Instruct",
  'max_new_tokens': 6000,
  'temperature': 0.1,
  'return_full_text': False
}
max_docs_per_prompt = 1  # Set the maximum number of documents per prompt

In [9]:
# Prepare input
tasks, target_summaries = prepare_input(
  dct_input = dataset,
  input_key = "document",
  target_key = "summary",
  max_inputs_per_prompt = max_docs_per_prompt
  )

In [10]:
# Check 1
print(tasks[0])
print()
print(target_summaries[0])
print()
print(target_summaries[:1])

Apply the task for the following instances:
1. The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.
Repair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.
Trains on the west coast mainline face disruption due to damage at the Lamington Viaduct.
Many businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.
First Minister Nicola Sturgeon visited the area to inspect the damage.
The waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.
Jeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.
However, she said more preventative work could have been carried out to ensure the retaining wall did not fail.
"It is difficult but I do think there is so much publicity for Dumfries and the Nith

In [11]:
# Check 2
output = execute_task(
    prompt = initial_prompt,
    tasks = tasks[:1],
    context = "",
    dct_params = dct_params
)


=== TASK EXECUTION INPUT START ===
Full prompt:

You are a summarization expert. Generate a concise summary.


    Output your answer concisely in the following XML format, using only these elements, and without repeating input information:

    <thoughts> Your understanding of the task and how do you plan to solve it </thoughts>
    <response> Your answer here </response>
    
Task: Apply the task for the following instances:
1. The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.
Repair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.
Trains on the west coast mainline face disruption due to damage at the Lamington Viaduct.
Many businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.
First Minister Nicola Sturgeon visited the area to inspect the damage.
The waters breached a retaining wall, flooding many commercial properties 

In [12]:
# Check 3
input_system_prompt = initial_prompt
tasks_check = tasks[:1]
targets_check = target_summaries[:1]
outputs = output[1]
memory = ""

output_refine = refine_prompt(
    input_system_prompt = input_system_prompt,
    tasks = tasks_check,
    memory = memory,
    outputs = outputs,
    targets = targets_check,
    dct_params = dct_params
)


=== PROMPT ENGINEERING INPUT START ===
Full prompt:

      You are a prompt engineering expert.

      1. Task:
      * Given a <input_system_prompt> for another LLM, the <tasks> that the LLM is trying to solve,
      the <generated_outputs> for that tasks following that input system prompt,
      the <target_outputs> that should've been generated, and the <memory> of previous recommendations
      that you have provided, propose an improved <input_system_prompt>.

      2. Notes:
      * The new base prompt proposed should be generic enough for approaching that task even with different input data.
      * Thus, do not use specific information about the input data within the Tasks.
      * The new base prompt can include aspects such as synthetic examples for improving it, text refinement, task clarification...
      * You have memory information on previous attempts: improvements previously proposed and the output obtained.

      3. Output format:
      * Output your answer concisel

In [13]:
# Check 4
final_outputs = iterative_task_execution(
    tasks=tasks[:1],
    initial_prompt=initial_prompt,
    target_outputs=target_summaries[:1],
    n_max_iter=2,
    dct_params=dct_params
)
print()
print("*"*50)
print(final_outputs)
print("*"*50)


=== ITERATION 1 START ===
**************************************************
current_system_prompt: 
You are a summarization expert. Generate a concise summary.

**************************************************

=== TASK EXECUTION INPUT START ===
Full prompt:

You are a summarization expert. Generate a concise summary.


    Output your answer concisely in the following XML format, using only these elements, and without repeating input information:

    <thoughts> Your understanding of the task and how do you plan to solve it </thoughts>
    <response> Your answer here </response>
    
Task: Apply the task for the following instances:
1. The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.
Repair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.
Trains on the west coast mainline face disruption due to damage at the Lamington Viaduct.
Many businesses and householders were affected by flood